In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Optimizing multiple objectives with Vertex AI Vizier

## Overview

This tutorial demonstrates Vertex AI Vizier multi-objective optimization. Multi-objective optimization is concerned with mathematical optimization problems involving more than one objective function to be optimized simultaneously.

Learn more about [Vertex AI Vizier](https://cloud.google.com/vertex-ai/docs/vizier/overview).

### Objective

In this tutorial, you learn how to use Vertex AI Vizier to optimize a multi-objective study.

The goal is to __`minimize`__ the objective metric:
   ```
   y1 = r*sin(theta)
   ```

and simultaneously __`maximize`__ the objective metric:
   ```
   y2 = r*cos(theta)
   ```

that you're going to evaluate over the parameter space:

   - __`r`__ in [0,1],

   - __`theta`__ in [0, pi/2]

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.


## Get started

### Install Vertex AI SDK for Python and other required packages


In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                               numpy==1.23.0 \
                               google-vizier

### Restart runtime (Colab only)

To use the newly installed packages, you must restart the runtime on Google Colab.

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

Authenticate your environment on Google Colab.


In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information

To get started using Vertex AI, you must have an existing Google Cloud project. Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
PROJECT_ID = "jc-gcp-project"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

Updated property [core/project].


### Import libraries and define constants

In [ ]:
pip show google-vizier

Name: google-vizier
Version: 0.1.24
Summary: Open Source Vizier: Distributed service framework for blackbox optimization and research.
Home-page: https://github.com/google/vizier
Author: Vizier Team
Author-email: oss-vizier-dev@google.com
License: Apache License 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: absl-py, attrs, googleapis-common-protos, grpcio, numpy, portpicker, protobuf, sqlalchemy
Required-by: 


In [ ]:
import datetime

from google.cloud import aiplatform
from google.cloud.aiplatform.vizier import Study, pyvizier

## Tutorial


This section defines some parameters and util methods to call Vertex Vizier APIs.

In [ ]:
# These will be automatically filled in.
STUDY_DISPLAY_NAME = "{}_study_{}".format(
    PROJECT_ID.replace("-", ""), datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
)
PARENT = "projects/{}/locations/{}".format(PROJECT_ID, LOCATION)

print("LOCATION: {}".format(LOCATION))
print("PARENT: {}".format(PARENT))

LOCATION: us-central1
PARENT: projects/jc-gcp-project/locations/us-central1


### Create the study configuration

The following is a sample study configuration, built as a hierarchical python dictionary. It's already filled out. Run the cell to configure the study.

In [ ]:
# Parameter Configuration
problem = pyvizier.StudyConfig()
problem.algorithm = pyvizier.Algorithm.RANDOM_SEARCH

# Objective Metrics
problem.metric_information.append(
    pyvizier.MetricInformation(name="y1", goal=pyvizier.ObjectiveMetricGoal.MINIMIZE)
)
problem.metric_information.append(
    pyvizier.MetricInformation(name="y2", goal=pyvizier.ObjectiveMetricGoal.MAXIMIZE)
)

# Defines the parameters configuration.
root = problem.search_space.select_root()
root.add_float_param("r", 0, 1.0, scale_type=pyvizier.ScaleType.LINEAR)
root.add_float_param("theta", 0, 1.57, scale_type=pyvizier.ScaleType.LINEAR)

ParameterConfigSelector(_selected=(ParameterConfig(_name='theta', _type=DOUBLE, _bounds=(0.0, 1.57), _feasible_values=None, _scale_type=LINEAR, _default_value=None, _external_type=INTERNAL, _children={}, _matching_parent_values=(), fidelity_config=None),))

### Create the study

Next, create the study, which you can subsequently run to optimize the two objectives.

In [ ]:
aiplatform.init(project=PROJECT_ID, location=LOCATION)
study = Study.create_or_load(display_name=STUDY_DISPLAY_NAME, problem=problem)

STUDY_ID = study.name
print("STUDY_ID: {}".format(STUDY_ID))

STUDY_ID: 789187478234


### Metric evaluation functions

Next, define some functions to evaluate the two objective metrics.

In [ ]:
import math


# r * sin(theta)
def Metric1Evaluation(r, theta):
    """Evaluate the first metric on the trial."""
    return r * math.sin(theta)


# r * cos(theta)
def Metric2Evaluation(r, theta):
    """Evaluate the second metric on the trial."""
    return r * math.cos(theta)


def CreateMetrics(trial_id, r, theta):
    print(("=========== Start Trial: [{}] =============").format(trial_id))

    # Evaluate both objective metrics for this trial
    y1 = Metric1Evaluation(r, theta)
    y2 = Metric2Evaluation(r, theta)
    print(
        "[r = {}, theta = {}] => y1 = r*sin(theta) = {}, y2 = r*cos(theta) = {}".format(
            r, theta, y1, y2
        )
    )
    measurement = pyvizier.Measurement()
    measurement.metrics["y1"] = y1
    measurement.metrics["y2"] = y2

    # Return the results for this trial
    return measurement

### Set configuration parameters for running trials

__`client_id`__: The identifier of the client requesting the suggestion. If multiple `SuggestTrialsRequests` have the same `client_id`, the service returns the identical suggested trial if the trial is `PENDING`, and provide a new trial if the last suggested trial is complete.

__`suggestion_count_per_request`__: The number of suggestions (trials) requested in a single request.

__`max_trial_id_to_stop`__: The number of trials to explore before stopping. It's set to 4 to shorten the time to run the code, so don't expect convergence. For convergence, it would likely need to be about 20 (a good rule of thumb is to multiply the total dimensionality by 10).


In [ ]:
worker_id = "worker1"  # @param {type: 'string'}
suggestion_count_per_request = 3  # @param {type: 'integer'}
max_trial_id_to_stop = 4  # @param {type: 'integer'}

print("worker_id: {}".format(worker_id))
print("suggestion_count_per_request: {}".format(suggestion_count_per_request))
print("max_trial_id_to_stop: {}".format(max_trial_id_to_stop))

worker_id: worker1
suggestion_count_per_request: 3
max_trial_id_to_stop: 4


### Run Vertex Vizier trials

Run the trials.

In [ ]:
while len(study.trials()) < max_trial_id_to_stop:
    trials = study.suggest(count=suggestion_count_per_request, worker=worker_id)

    for suggested_trial in trials:
        measurement = CreateMetrics(
            suggested_trial.name,
            suggested_trial.parameters["r"].value,
            suggested_trial.parameters["theta"].value,
        )
        suggested_trial.add_measurement(measurement=measurement)
        suggested_trial.complete(measurement=measurement)

INFO:google.cloud.aiplatform.vizier.study:Suggest Study study backing LRO: projects/78565683329/locations/us-central1/studies/789187478234/operations/789187478234_worker1_1
INFO:google.cloud.aiplatform.vizier.study:<class 'google.cloud.aiplatform_v1.services.vizier_service.client.VizierServiceClient'>
INFO:google.cloud.aiplatform.vizier.study:Study study suggested. Resource name: projects/78565683329/locations/us-central1/studies/789187478234


=========== Start Trial: [1] =============
[r = 0.7337310888556299, theta = 0.5042108348071791] => y1 = r*sin(theta) = 0.35447769281552166, y2 = r*cos(theta) = 0.6424226615316841
=========== Start Trial: [2] =============
[r = 0.645009628394043, theta = 0.5379870181391783] => y1 = r*sin(theta) = 0.330508351113888, y2 = r*cos(theta) = 0.553896786924243
=========== Start Trial: [3] =============
[r = 0.8619851526090625, theta = 0.23858922136157312] => y1 = r*sin(theta) = 0.2037147160020711, y2 = r*cos(theta) = 0.8375671422654212


INFO:google.cloud.aiplatform.vizier.study:Suggest Study study backing LRO: projects/78565683329/locations/us-central1/studies/789187478234/operations/789187478234_worker1_2
INFO:google.cloud.aiplatform.vizier.study:<class 'google.cloud.aiplatform_v1.services.vizier_service.client.VizierServiceClient'>
INFO:google.cloud.aiplatform.vizier.study:Study study suggested. Resource name: projects/78565683329/locations/us-central1/studies/789187478234


=========== Start Trial: [4] =============
[r = 0.8832965932872785, theta = 0.09696319026052437] => y1 = r*sin(theta) = 0.08551311145223091, y2 = r*cos(theta) = 0.879147529987243
=========== Start Trial: [5] =============
[r = 0.7154259464410591, theta = 1.4670803138442237] => y1 = r*sin(theta) = 0.711581472039731, y2 = r*cos(theta) = 0.07406816786484281
=========== Start Trial: [6] =============
[r = 0.4686904970711221, theta = 0.24139085048789743] => y1 = r*sin(theta) = 0.11204204842398328, y2 = r*cos(theta) = 0.45510148475887585


### List the optimal solutions

The `optimal_trials()` method returns the Pareto-optimal trials for a multi-objective study or the optimal trials for a single-objective study. If multiple objectives are defined in previous steps, Pareto-optimal trials are returned.

In [ ]:
optimal_trials = study.optimal_trials()
print("optimal_trials: {}".format(optimal_trials))

optimal_trials: [<google.cloud.aiplatform.vizier.trial.Trial object at 0x7e3abb8958a0> 
resource name: projects/78565683329/locations/us-central1/studies/789187478234/trials/4]


## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial. You can also manually delete resources that you created by running the following code.

In [ ]:
study.delete()